In [1]:
from sklearn.metrics.pairwise import euclidean_distances, cosine_similarity
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np
import os
import json
import seaborn as sn
import matplotlib.pyplot as plt
import pandas as pd
import create_directory

# Load Data

In [2]:
data_path = os.path.join(create_directory.marvel_data_dir, "distance_face_recognition/fixed_threshold")

In [3]:
# label2idx
with open(os.path.join(create_directory.recognition_model_dir, 'label2idx.json')) as json_file:
    label2idx = json.load(json_file)

# Face recognition

## Cài đặt hàm

In [4]:
# Các hàm nhận diện khuôn mặt
def nearest_recognition(feat, train_labels, train_feats, metric, threshold):
    candidate_label = []
    candidate_score = []
    for label, vector in zip(train_labels, train_feats):
        if metric=='euclidean_distance':
            metric_score = euclidean_distances(np.array(feat).reshape(1, -1), np.array(vector).reshape(1, -1))[0][0]
            if metric_score<threshold:
                candidate_label.append(label)
                candidate_score.append(metric_score)
        elif metric=="cosine_similarity":
            metric_score = cosine_similarity(np.array(feat).reshape(1, -1), np.array(vector).reshape(1, -1))[0][0]
            if metric_score>threshold:
                candidate_label.append(label)
                candidate_score.append(metric_score)
    if len(candidate_label)==0:
        return label2idx["Unknown"]
    else:
        if metric=='euclidean_distance':
            return candidate_label[candidate_score.index(min(candidate_score))]
        if metric=='cosine_similarity':
            return candidate_label[candidate_score.index(max(candidate_score))]

def naive_neighbour_recognition(feat, train_labels, train_feats, metric, threshold):
    candidate_label = []
    candidate_score = []
    for label, vector in zip(train_labels, train_feats):
        if metric == "euclidean_distance":
            metric_score = euclidean_distances(np.array(feat).reshape(1, -1), np.array(vector).reshape(1, -1))[0][0]
            if metric_score<threshold:
                candidate_label.append(label)
                candidate_score.append(metric_score)
        elif metric=="cosine_similarity":
            metric_score = cosine_similarity(np.array(feat).reshape(1, -1), np.array(vector).reshape(1, -1))[0][0]
            if metric_score>threshold:
                candidate_label.append(label)
                candidate_score.append(metric_score)
    if len(candidate_label)==0:
        return label2idx["Unknown"]
    else:
        df_candidate=pd.DataFrame({'Label': candidate_label})
        df_candidate_count = pd.DataFrame(df_candidate["Label"].value_counts().to_dict().items(), columns=['Label','Count'])
        list_candidate = list(df_candidate_count[df_candidate_count['Count']==max(df_candidate_count['Count'])]['Label'])
        if len(list_candidate) == 1:
            return max(candidate_label, key = candidate_label.count)
        else:
            df_candidate_score=pd.DataFrame({'Label': candidate_label, 'Score': candidate_score})
            best_candidate = df_candidate_score[df_candidate_score['Label'].isin(list_candidate)].groupby('Label').mean().to_dict()['Score']
            if metric=='euclidean_distance':
                return min(best_candidate, key = best_candidate.get)
            elif metric=='cosine_similarity':
                return max(best_candidate, key = best_candidate.get)

def weight_neighbour_recognition(feat, train_labels, train_feats, metric, threshold):
    candidate_label = []
    candidate_score = []
    for label, vector in zip(train_labels, train_feats):
        if metric=='euclidean_distance':
            score = euclidean_distances(np.array(feat).reshape(1, -1), np.array(vector).reshape(1, -1))[0][0]
            if score<threshold:
                candidate_label.append(label)
                candidate_score.append(score)
        elif metric=='cosine_similarity':
            score = cosine_similarity(np.array(feat).reshape(1, -1), np.array(vector).reshape(1, -1))[0][0]
            if score>threshold:
                candidate_label.append(label)
                candidate_score.append(score)
    if len(candidate_label)==0:
        return label2idx["Unknown"]
    else:
        weight = {}
        if metric=='euclidean_distance':
            for i in range(len(candidate_label)):
                can = candidate_label[i]
                dis = candidate_score[i]
                if can not in weight:
                    weight[can]=0
                weight[can]+=1/dis
            return max(weight, key = weight.get)
        elif metric=='cosine_similarity':
            for i in range(len(candidate_label)):
                can = candidate_label[i]
                sim = candidate_score[i]
                if can not in weight:
                    weight[can]={}
                    weight[can]['total']=0
                    weight[can]['count']=1
                weight[can]['total']+=sim
                weight[can]['count']+=1
            mean={}
            for label, info in weight.items():
                mean[label]=info['total']/info['count']
            return max(mean, key = mean.get)

In [5]:
# Lấy kết quả dự đoán ứng với các threshold tương ứng
threshold_list = np.arange(0.1, 1, 0.05)
def test_threshold(threshold_list, x_train, x_test, y_train, type, metric):
    recognition_result = {}
    for i in threshold_list:
        print("Threshold: "+str(i))
        if type=="nearest":
            recognition_result[i] = [nearest_recognition(j, y_train, x_train, metric, i) for j in x_test]
        if type=="naive_neighbour":
            recognition_result[i] = [naive_neighbour_recognition(j, y_train, x_train, metric, i) for j in x_test]
        if type=="weight_neighbour":
            recognition_result[i] = [weight_neighbour_recognition(j, y_train, x_train, metric, i) for j in x_test]
    return recognition_result

def evaluation(true, pred):
    fa = 0  # False accept
    wa = 0  # Wrong answer
    fr = 0  # False reject
    accept = 0
    reject = 0

    for (i, j) in zip(true, pred):
        # Hệ thống nhận diện khuôn mặt đó có trong database
        if j != label2idx["Unknown"]:
            accept+=1
            # Hệ thống nhận diện khuôn mặt Unknown thành khuôn mặt trong database
            if i == label2idx["Unknown"]:
                fa+=1
            else:
                # Hệ thống nhận diện nhầm khuôn mặt trong database
                if i!=j:
                    wa+=1
        else:
            reject+=1
            if i != label2idx["Unknown"]:
                fr+=1
    # Mong muốn giảm fa, wa
    return (fa, wa, fr, accept, reject)

In [6]:
# Plot confusion matrix
idx2label = {y: x for x, y in label2idx.items()}
def show_result(result, y_test, threshold):
    pred = result[threshold]
    fa, wa, fr, accept, reject = evaluation(y_test, pred)
    acc = accuracy_score(y_test, pred)
    print("Threshold = "+str(threshold))
    print("Accuracy: {}, Accept: {}, False Accept: {}, Wrong Recognition: {}, Reject: {}, False Reject: {}".format(acc, accept, fa, wa, reject, fr))
    print("")
    array = confusion_matrix(y_test, pred)
    df_cm = pd.DataFrame(array, index = [idx2label[i] for i in range(7)], 
                        columns = [idx2label[i] for i in range(7)])
    sn.heatmap(df_cm, annot=True, annot_kws={"size": 16}) # font size
    plt.title("Confusion matrix with threshold = "+str(threshold))
    plt.show()

# Save result
def save_result(result, y_test, save_path):
    thresh_list, fa_list, wa_list, fr_list, accept_list, reject_list, accuracy_list = [], [], [], [], [], [], []
    for threshold, pred in result.items():
        fa, wa, fr, accept, reject = evaluation(y_test, pred)
        acc = accuracy_score(y_test, pred)
        thresh_list.append(threshold)
        fa_list.append(fa)
        wa_list.append(wa)
        fr_list.append(fr)
        accept_list.append(accept)
        reject_list.append(reject)
        accuracy_list.append(acc)
    dict = {'threshold': thresh_list, 'accept': accept_list, 'fa': fa_list, 'wa': wa_list, 
            'reject': reject_list, 'fr': fr_list, 'accuracy': accuracy_list} 
    df = pd.DataFrame(dict)
    df.to_csv(save_path, index=False)

In [7]:
def process(threshold_list, x_train, x_test, y_train, y_test, type, metric, save_path):
    recognition_result = test_threshold(threshold_list, x_train, x_test, y_train, type, metric)
    save_result(recognition_result, y_test, save_path)

save_path = os.path.join(create_directory.result_dir, 'test_threshold/fixed_threshold')
if not os.path.exists(save_path):
    os.makedirs(save_path)
sample_list = [0,1,2,3]
def process_sample(data_path, sample_list, threshold_list, type, metric, save_path):
    if not os.path.exists(os.path.join(save_path, metric, type)):
        os.makedirs(os.path.join(save_path, metric, type))
    for i in sample_list:
        print('Sample {}'.format(i))
        with open(os.path.join(data_path, "train_sample_{}_feature.json".format(i)), "r") as outfile:
            x_train = json.load(outfile)
        with open(os.path.join(data_path, "test_feature.json"), "r") as outfile:
            x_test = json.load(outfile)
        with open(os.path.join(data_path, "train_sample_{}_label_idx.json".format(i)), "r") as outfile:
            y_train = json.load(outfile)
        with open(os.path.join(data_path, "test_label_idx.json"), "r") as outfile:
            y_test = json.load(outfile)
        process(threshold_list, x_train, x_test, y_train, y_test, type, metric,
                os.path.join(save_path, metric, type, 'sample_{}.csv'.format(i)))        

## Euclidean distance

### Nearest candidate

In [8]:
process_sample(data_path, sample_list, threshold_list, 'nearest', 'euclidean_distance', save_path)

Sample 0
Threshold: 0.1
Threshold: 0.15000000000000002
Threshold: 0.20000000000000004
Threshold: 0.25000000000000006
Threshold: 0.30000000000000004
Threshold: 0.3500000000000001
Threshold: 0.40000000000000013
Threshold: 0.45000000000000007
Threshold: 0.5000000000000001
Threshold: 0.5500000000000002
Threshold: 0.6000000000000002
Threshold: 0.6500000000000001
Threshold: 0.7000000000000002
Threshold: 0.7500000000000002
Threshold: 0.8000000000000002
Threshold: 0.8500000000000002
Threshold: 0.9000000000000002
Threshold: 0.9500000000000003
Sample 1
Threshold: 0.1
Threshold: 0.15000000000000002
Threshold: 0.20000000000000004
Threshold: 0.25000000000000006
Threshold: 0.30000000000000004
Threshold: 0.3500000000000001
Threshold: 0.40000000000000013
Threshold: 0.45000000000000007
Threshold: 0.5000000000000001
Threshold: 0.5500000000000002
Threshold: 0.6000000000000002
Threshold: 0.6500000000000001
Threshold: 0.7000000000000002
Threshold: 0.7500000000000002
Threshold: 0.8000000000000002
Threshold:

### Naive Neighbour candidates

In [ ]:
process_sample(data_path, sample_list, threshold_list, 'naive_neighbour', 'euclidean_distance', save_path)

Sample 0
Threshold: 0.1
Threshold: 0.15000000000000002
Threshold: 0.20000000000000004
Threshold: 0.25000000000000006
Threshold: 0.30000000000000004
Threshold: 0.3500000000000001
Threshold: 0.40000000000000013
Threshold: 0.45000000000000007
Threshold: 0.5000000000000001
Threshold: 0.5500000000000002
Threshold: 0.6000000000000002
Threshold: 0.6500000000000001
Threshold: 0.7000000000000002
Threshold: 0.7500000000000002
Threshold: 0.8000000000000002
Threshold: 0.8500000000000002
Threshold: 0.9000000000000002
Threshold: 0.9500000000000003
Sample 1
Threshold: 0.1
Threshold: 0.15000000000000002
Threshold: 0.20000000000000004
Threshold: 0.25000000000000006
Threshold: 0.30000000000000004
Threshold: 0.3500000000000001
Threshold: 0.40000000000000013
Threshold: 0.45000000000000007
Threshold: 0.5000000000000001
Threshold: 0.5500000000000002
Threshold: 0.6000000000000002
Threshold: 0.6500000000000001
Threshold: 0.7000000000000002
Threshold: 0.7500000000000002
Threshold: 0.8000000000000002
Threshold:

### Weight Neighbour candidates

In [ ]:
process_sample(data_path, sample_list, threshold_list, 'weight_neighbour', 'euclidean_distance', save_path)

Sample 0
Threshold: 0.1
Threshold: 0.15000000000000002
Threshold: 0.20000000000000004
Threshold: 0.25000000000000006
Threshold: 0.30000000000000004
Threshold: 0.3500000000000001
Threshold: 0.40000000000000013
Threshold: 0.45000000000000007
Threshold: 0.5000000000000001
Threshold: 0.5500000000000002
Threshold: 0.6000000000000002
Threshold: 0.6500000000000001
Threshold: 0.7000000000000002
Threshold: 0.7500000000000002
Threshold: 0.8000000000000002
Threshold: 0.8500000000000002
Threshold: 0.9000000000000002
Threshold: 0.9500000000000003
Sample 1
Threshold: 0.1
Threshold: 0.15000000000000002
Threshold: 0.20000000000000004
Threshold: 0.25000000000000006
Threshold: 0.30000000000000004
Threshold: 0.3500000000000001
Threshold: 0.40000000000000013
Threshold: 0.45000000000000007
Threshold: 0.5000000000000001
Threshold: 0.5500000000000002
Threshold: 0.6000000000000002
Threshold: 0.6500000000000001
Threshold: 0.7000000000000002
Threshold: 0.7500000000000002
Threshold: 0.8000000000000002
Threshold:

## Cosnine similarity

### Nearest candidate

In [ ]:
process_sample(data_path, sample_list, threshold_list, 'nearest', 'cosine_similarity', save_path)

Sample 0
Threshold: 0.1
Threshold: 0.15000000000000002
Threshold: 0.20000000000000004
Threshold: 0.25000000000000006
Threshold: 0.30000000000000004
Threshold: 0.3500000000000001
Threshold: 0.40000000000000013
Threshold: 0.45000000000000007
Threshold: 0.5000000000000001
Threshold: 0.5500000000000002
Threshold: 0.6000000000000002
Threshold: 0.6500000000000001
Threshold: 0.7000000000000002
Threshold: 0.7500000000000002
Threshold: 0.8000000000000002
Threshold: 0.8500000000000002
Threshold: 0.9000000000000002
Threshold: 0.9500000000000003
Sample 1
Threshold: 0.1
Threshold: 0.15000000000000002
Threshold: 0.20000000000000004
Threshold: 0.25000000000000006
Threshold: 0.30000000000000004
Threshold: 0.3500000000000001
Threshold: 0.40000000000000013
Threshold: 0.45000000000000007
Threshold: 0.5000000000000001
Threshold: 0.5500000000000002
Threshold: 0.6000000000000002
Threshold: 0.6500000000000001
Threshold: 0.7000000000000002
Threshold: 0.7500000000000002
Threshold: 0.8000000000000002
Threshold:

KeyboardInterrupt: 

### Naive neighbour candidate

In [ ]:
process_sample(data_path, sample_list, threshold_list, 'naive_neighbour', 'cosine_similarity', save_path)

Threshold: 0.1
Threshold: 0.15000000000000002
Threshold: 0.20000000000000004
Threshold: 0.25000000000000006
Threshold: 0.30000000000000004
Threshold: 0.3500000000000001
Threshold: 0.40000000000000013
Threshold: 0.45000000000000007
Threshold: 0.5000000000000001
Threshold: 0.5500000000000002
Threshold: 0.6000000000000002
Threshold: 0.6500000000000001
Threshold: 0.7000000000000002
Threshold: 0.7500000000000002
Threshold: 0.8000000000000002
Threshold: 0.8500000000000002
Threshold: 0.9000000000000002
Threshold: 0.9500000000000003


### Weight Neighbour candidates

In [ ]:
process_sample(data_path, sample_list, threshold_list, 'weight_neighbour', 'cosine_similarity', save_path)

Threshold: 0.1
Threshold: 0.15000000000000002
Threshold: 0.20000000000000004
Threshold: 0.25000000000000006
Threshold: 0.30000000000000004
Threshold: 0.3500000000000001
Threshold: 0.40000000000000013
Threshold: 0.45000000000000007
Threshold: 0.5000000000000001
Threshold: 0.5500000000000002
Threshold: 0.6000000000000002
Threshold: 0.6500000000000001
Threshold: 0.7000000000000002
Threshold: 0.7500000000000002
Threshold: 0.8000000000000002
Threshold: 0.8500000000000002
Threshold: 0.9000000000000002
Threshold: 0.9500000000000003
